## Desafio Alura

Você foi contratado(a) como pessoa cientista de dados para trabalhar em um banco digital internacional chamado Alura Cash. Na primeira reunião do seu novo trabalho, a diretoria financeira informa que, recorrentemente, estão surgindo pessoas inadimplentes após a liberação de créditos. Portanto, é solicitada uma solução para que seja possível diminuir as perdas financeiras por conta de pessoas mutuarias que não quitam suas dívidas.

Como cientista de dados, você sugere um estudo das informações financeiras e de solicitação de empréstimo para encontrar padrões que possam indicar uma possível inadimplência.

Desse modo, você solicita um conjunto de dados que contenha as informações de clientes, da solicitação de empréstimo, do histórico de crédito, bem como se a pessoa mutuaria é inadimplente ou não. Com esses dados, você sabe que consegue modelar um classificador capaz de encontrar potenciais clientes inadimplentes e solucionar o problema do Alura Cash.

Por fim, você vai utilizar o GitHub e desenvolver um portfólio focado em Data Science, Data Analytics e Machine Learning.

Link do repositório: [https://github.com/Mirlaa/Challenge-Data-Science-1ed](https://github.com/Mirlaa/Challenge-Data-Science-1ed)


### Atividades realizadas (Semana 1)
- Instalar MySQL Workbench e importar **database dump**.
- Analisar quais os tipos de dados.
- Verificar quais são as inconsistências nos dados.
- Entender quais informações o conjunto de dados possui.
- Verificar quais são as inconsistências nos dados.
- Corrigir as inconsistências nos dados.
- Unir as tabelas de dados de acordo com os IDs.
- Traduzir as colunas.
- Exportar a tabela de dados unidos como csv.

#### Instalar MySQL Workbench e importar **database dump**
![](import_dump.png "Title")

#### Analisar quais os tipos de dados

In [2]:
# bibliotecas usadas:
import mysql.connector as sql # fazer a conexão com o MySQL
import pandas as pd # manipulação dos dados

In [1]:
# opcional
# Ocutar alguns alertas:
# (UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or
# sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy)
import warnings
warnings.filterwarnings('ignore')

In [4]:
# conexão
db_connection = sql.connect(host='127.0.0.1', database='aluracash', user='root', password='senha123')

In [5]:
# testar conexao
table_id = pd.read_sql('SELECT * FROM id', con=db_connection)
table_id

,person_id,loan_id,cb_id
0,JAz2tRvQ-14cy,pjEu6snv-4t33,FIgNSkrG-wen7
1,MZSefq9m-loYS,Mfn0Csx4-LGM3,QPDJUQfQ-DJqK
2,cRItaaPB-pRvT,Ks4nz74P-z6Iq,hzAXqSDh-ay7Q
3,5jTDCfXS-zaAq,ZFlwW08i-I3Al,LiAivrS4-J5DE
4,Hbz9PtI3-tOFs,bFMFchcv-UwQW,PkBixMJ3-3bad
...,...,...,...
14947,Fq62KEaz-3xZS,oE7OxjOb-M0os,3oTt3H7Y-zvaJ
14948,3Nc47pRD-sOoq,ey2oe45x-gRBl,9pilhQPW-zkr4
14949,Kpzf8Ll0-r5tA,mvTg1J3Z-I6HB,vahJw1oD-SxUA
14950,o8MJQJNE-Finv,ffr3kplw-61FF,GhYJdT43-IBBK


#### Traduzir as colunas
Foram criadas **view** com as colunas e informações traduzidas.
Todas as **view** receberam o nome original das tabelas + o sulfixo ``_t``.

view ``dados_mutuarios_t``:

    SELECT 
        `dados_mutuarios`.`person_id` AS `PESSOA`,
        `dados_mutuarios`.`person_age` AS `IDADE`,
        `dados_mutuarios`.`person_income` AS `RENDA_ANUAL`,
        (CASE
            WHEN (`dados_mutuarios`.`person_home_ownership` = 'Rent') THEN 'Alugada'
            WHEN (`dados_mutuarios`.`person_home_ownership` = 'Mortgage') THEN 'Hipotecada'
            WHEN (`dados_mutuarios`.`person_home_ownership` = 'Own') THEN 'Própria'
            WHEN (dados_mutuarios.person_home_ownership = 'Other') THEN 'Outro'
            ELSE dados_mutuarios.person_home_ownership
        END) AS SITUACAO_PROPRIEDADE,
        dados_mutuarios.person_emp_length AS TRABALHO_ANOS
    FROM
        dados_mutuarios



view ``emprestimos_t``:

    SELECT 
        `emprestimos`.`loan_id` AS `EMPRESTIMO`,
        (CASE
            WHEN (`emprestimos`.`loan_intent` = 'Personal') THEN 'Pessoal'
            WHEN (`emprestimos`.`loan_intent` = 'Education') THEN 'Educativo'
            WHEN (`emprestimos`.`loan_intent` = 'Medical') THEN 'Médico'
            WHEN (emprestimos.loan_intent = 'Venture') THEN 'Empreendimento'
            WHEN (emprestimos.loan_intent = 'Homeimprovement') THEN 'Melhora do lar'
            WHEN (emprestimos.loan_intent = 'Debtconsolidation') THEN 'Pagamento de débitos'
            ELSE emprestimos.loan_intent
        END) AS MOTIVO_EMPRESTIMO,
        emprestimos.loan_grade AS PONTUACAO,
        emprestimos.loan_amnt AS VALOR_EMPRESTIMO,
        emprestimos.loan_int_rate AS TAXA_JUROS,
        emprestimos.loan_status AS NIVEL_INADIMPLENCIA,
        emprestimos.loan_percent_income AS EMPRESTIMO_PERC_RENDA_ANUAL
    FROM
        emprestimos



view ``emprestimos_t``:

    SELECT 
        `historicos_banco`.`cb_id` AS `SOLICITACAO`,
        (CASE
            WHEN (`historicos_banco`.`cb_person_default_on_file` = 'Y') THEN 'SIM'
            WHEN (`historicos_banco`.`cb_person_default_on_file` = 'N') THEN 'NÃO'
            ELSE historicos_banco.cb_person_default_on_file
        END) AS INADIMPLENTE,
        historicos_banco.cb_person_cred_hist_length AS ANOS_PRIMEIRO_CREDITO
    FROM
        historicos_banco


Não foi necessário criar uma **view** para a tabela ``id``.


In [29]:
# carregar as outras tabelas
table_dados_mutuarios = pd.read_sql('SELECT * FROM dados_mutuarios_t', con=db_connection)
table_emprestimos = pd.read_sql('SELECT * FROM emprestimos_t', con=db_connection)
table_historicos_banco = pd.read_sql('SELECT * FROM historicos_banco_t', con=db_connection)

#### Analisar quais os tipos de dados

In [30]:
# id
print('========== id ==========')
print(table_id.info())
print('\n========== table_dados_mutuarios ==========')
print(table_dados_mutuarios.info())
print('\n========== table_emprestimos ==========')
print(table_emprestimos.info())
print('\n========== table_historicos_banco ==========')
print(table_historicos_banco.info())


========== id ==========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14952 entries, 0 to 14951
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  14952 non-null  object
 1   loan_id    14952 non-null  object
 2   cb_id      14952 non-null  object
dtypes: object(3)
memory usage: 350.6+ KB
None

========== table_dados_mutuarios ==========
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32662 entries, 0 to 32661
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PESSOA                32662 non-null  object 
 1   IDADE                 32445 non-null  float64
 2   RENDA_ANUAL           32662 non-null  int64  
 3   SITUACAO_PROPRIEDADE  32662 non-null  object 
 4   TRABALHO_ANOS         32662 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.2+ MB
None

========== table_emprestimos ==========
<class 

#### Entender quais informações o conjunto de dados possui

A tabela **id** possui as id para o relacionamento com as outras tabelas.


person_id = PESSOA

loan_id = EMPRESTIMO

cb_id = SOLICITACAO

In [32]:
table_id.head()

,person_id,loan_id,cb_id
0,JAz2tRvQ-14cy,pjEu6snv-4t33,FIgNSkrG-wen7
1,MZSefq9m-loYS,Mfn0Csx4-LGM3,QPDJUQfQ-DJqK
2,cRItaaPB-pRvT,Ks4nz74P-z6Iq,hzAXqSDh-ay7Q
3,5jTDCfXS-zaAq,ZFlwW08i-I3Al,LiAivrS4-J5DE
4,Hbz9PtI3-tOFs,bFMFchcv-UwQW,PkBixMJ3-3bad


A tabela **dados_mutuarios_t** possui os dados pessoas, e apresenta tipos como **INT**, **FLOAT** e **STRING**.

In [37]:
table_dados_mutuarios.head()

,PESSOA,IDADE,RENDA_ANUAL,SITUACAO_PROPRIEDADE,TRABALHO_ANOS
0,tad8vkTn-vp4l,33.0,48000,Alugada,17.0
1,dEILQqp2-lgLx,24.0,75000,Própria,0.0
2,Az11sWWw-SdFA,29.0,28800,Alugada,5.0
3,YDovEwBH-0eek,23.0,75000,Hipotecada,7.0
4,lNsbfwlP-gzOE,26.0,35000,Alugada,0.0


A tabela **table_emprestimos_t** possui os dados dos empréstimos, e apresenta tipos como **INT**, **FLOAT** e **STRING**.

In [41]:
table_emprestimos.head()

,EMPRESTIMO,MOTIVO_EMPRESTIMO,PONTUACAO,VALOR_EMPRESTIMO,TAXA_JUROS,NIVEL_INADIMPLENCIA,EMPRESTIMO_PERC_RENDA_ANUAL
0,NMgFWFWN-2yGG,Melhora do lar,C,20000.0,13.22,0,0.22
1,08Ts3XSa-KkvE,Empreendimento,C,6000.0,13.49,0,0.04
2,xo79pAME-elmp,Pessoal,A,8000.0,6.17,0,0.13
3,IB6lBS2h-nWhz,Melhora do lar,B,9600.0,10.08,0,0.16
4,CONLtJHn-1yde,Médico,A,6500.0,8.49,0,0.30


A tabela **historicos_banco_t** possui os dados dos empréstimos, e apresenta tipos como **FLOAT** e **STRING**.

In [40]:
table_historicos_banco.head()

,SOLICITACAO,INADIMPLENTE,ANOS_PRIMEIRO_CREDITO
0,ZNWGR0Ap-FDt2,NÃO,9.0
1,wqALDH4x-mnzE,NÃO,7.0
2,57O09sJw-NBKj,NÃO,6.0
3,pH50QiTJ-0jqz,SIM,10.0
4,3voX5CIK-tCxE,NÃO,12.0


#### Verificar/Corrigir as inconsistências nos dados

In [68]:
# id
print('Nulos:')
print(table_id.isnull().sum())
print()
print('Vazios:')
print((table_id == '').sum())


Nulos:
person_id    0
loan_id      0
cb_id        0
dtype: int64

Vazios:
person_id    0
loan_id      0
cb_id        0
dtype: int64


In [72]:
# table_dados_mutuarios
print('Nulos:')
print(table_emprestimos.isnull().sum())
print()
print('Vazios:')
print((table_emprestimos == '').sum())

Nulos:
EMPRESTIMO                       0
MOTIVO_EMPRESTIMO                0
PONTUACAO                        0
VALOR_EMPRESTIMO               232
TAXA_JUROS                       0
NIVEL_INADIMPLENCIA              0
EMPRESTIMO_PERC_RENDA_ANUAL    217
dtype: int64

Vazios:
EMPRESTIMO                       0
MOTIVO_EMPRESTIMO              238
PONTUACAO                      230
VALOR_EMPRESTIMO                 0
TAXA_JUROS                       0
NIVEL_INADIMPLENCIA              0
EMPRESTIMO_PERC_RENDA_ANUAL      0
dtype: int64


In [ ]:
# table_dados_mutuarios
print('Nulos:')
print(table_emprestimos.isnull().sum())
print()
print('Vazios:')
print((table_emprestimos == '').sum())

In [73]:
# table_dados_mutuarios
print('Nulos:')
print(table_historicos_banco.isnull().sum())
print()
print('Vazios:')
print((table_historicos_banco == '').sum())

Nulos:
SOLICITACAO              0
INADIMPLENTE             0
ANOS_PRIMEIRO_CREDITO    1
dtype: int64

Vazios:
SOLICITACAO                0
INADIMPLENTE             367
ANOS_PRIMEIRO_CREDITO      0
dtype: int64


### Atividades realizadas (Semana 1)
- [OK] Instalar MySQL Workbench e importar **database dump**.
- [OK] Analisar quais os tipos de dados.
- [OK] Verificar quais são as inconsistências nos dados.
- [OK] Entender quais informações o conjunto de dados possui.
- [OK] Verificar quais são as inconsistências nos dados.
- Corrigir as inconsistências nos dados.
- Unir as tabelas de dados de acordo com os IDs.
- [OK] Traduzir as colunas.
- Exportar a tabela de dados unidos como csv.